# Summarizing texts

Large Language Models are very good at processing large amounts of text and paying attention to the most important parts. We can use this ability to summarize texts.

In this notebook, we will use an LLM to summarize the [Federalist Papers](https://en.wikipedia.org/wiki/The_Federalist_Papers).


## Loading our API key

At this point you should have set up a file named `secrets.env` with your OpenAI API key. We will now use a lightweight Python package called `dotenv` to read in this file and set its contents as environment variables:


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv("../secrets.env")

os.getenv(
    "OPENAI_API_KEY"
) is not None  # Do not print the key itself! We want to keep it secret

## Loading text documents


If you are running this on Dartmouth's JupyterHub, the documents are already stored as individual text files for you as a dataset online. If you are running this anywhere else, [download and extract the dataset](https://git.dartmouth.edu/lib-digital-strategies/RDS/datasets/federalist-papers-dataset/-/archive/main/federalist-papers-dataset-main.zip) and change the path in the next cell accordingly.


In [ ]:
from pathlib import Path

docs_dir = Path(
    "~/shared/RR-workshop-data/federalist-papers-dataset/split"
).expanduser()

We could use regular Python to read each of the papers like so:


In [ ]:
with open(docs_dir / "federalist_1.txt") as f:
    doc = f.read()

print(doc[:200])

However, we can benefit from _LangChain_'s document loaders, which reads all the files in the directory with just a few lines of code:


In [ ]:
from langchain.document_loaders import DirectoryLoader

doc_loader = DirectoryLoader(docs_dir, show_progress=True)
docs = doc_loader.load()

## Creating a summary

### Stuffing

The most straightforward way to ask a model to summarize a piece of text, is to send the entire text and ask the LLM to summarize it. This approach is called _stuffing_:


In [ ]:
from langchain.chat_models import ChatOpenAI
from openai.error import InvalidRequestError

llm = ChatOpenAI(model="gpt-3.5-turbo")

try:
    for doc in docs[:3]:
        print("-" * 30)
        print(
            llm.predict(
                "Summarize the following text in 200 words: \n" + doc.page_content
            )
        )
except InvalidRequestError as e:
    print(e._message)

As we can see, the model successfully summarizes the first document, but fails to summarize the second one.

The reason for this is the length of the text: A Large Language Model cannot process arbitrarily long input strings. The maximum number of tokens (one token ~ 3/4 word) an LLM can take into account while producing the next output token is called the _context window_. You can think of this as the "attention span" of the model.

The context window for the [regular GPT 3.5 turbo is 4097](https://platform.openai.com/docs/models/gpt-3-5), but there is model called `'gpt-3.5-turbo-16k'` that offers four times the context window. So let's switch to that and try again:


In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

for doc in docs[:3]:
    print("-" * 30)
    print(
        llm.predict("Summarize the following text in 200 words: \n" + doc.page_content)
    )

### Using a chain

Since summarizing documents is a popular use-case, _LangChain_ offers a pre-configured chain `StuffDocumentsChain`:


In [ ]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Define prompt
prompt_template = "Write a concise summary of the following:{text}"

prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(docs[:1]))

### Convenience wrapper

We can simplify this even more by using the convenient wrapper `load_summarize_chain()`, which comes with a predefined prompt suitable for this task:


In [ ]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm=llm, chain_type="stuff")

chain.run(docs[:1])

## Map-reduce

Stuffing works great for shorter texts and models with a large context window. But what if we want to process longer documents, or a much larger number of documents?

One technique to consider in this case is _Map-reduce_. It consists of two steps:

1. **Map**: Chunk the texts to summarize into smaller units (e.g., shorter documents, paragraphs) and summarize each chunk
2. **Reduce**: Treat the summaries like new documents and summarize the summaries with the LLM, using either Stuffing or another iteration of Map-Reduce

Just like with Stuffing, _LangChain_ also supports Map-reduce with specialized objects that facilitate the process.

We split the total map-reduce chain into a map chain and a reduce chain. These chains consist of a model and a prompt:


In [ ]:
# Map
map_template = "Write a concise summary of the following: {docs}"
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = "Distill the following summaries into a single summary of the main topics: {doc_summaries}"
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

It is not guaranteed that the sum of all the outputs of the _map_ chain fit into the context window of the _reduce_ chain. We therefore may have to run _map-reduce_ iteratively. We could implement that manually, but _LangChain_ offers some convenient components to help with this.

In addition to these LLm chains, we need two components:

1. An object that handles the stuffing part preceding the _map_ step
2. An object that handles the reduce step

We can use _LangChain_'s `StuffDocumentsChain` to do the first task:


In [ ]:
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_summaries",  # This is the name of the prompt variable to fill with the stuffed documents
)

The second part can be handled by a `ReduceDocumentsChain`. Since it may have to run multiple iterations, this object needs to have a way to combine the input documents, in case they do not fit into the context window as one long string. This is defined by the object we defined in the previous step:


In [ ]:
from langchain.chains import ReduceDocumentsChain

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is how the documents get combined
    combine_documents_chain=combine_documents_chain,
    # This is the chain to use to deal with documents that do not fit into the context window
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=2000,
)

Finally, we can put it all together in a `MapReduceDocumentsChain`:


In [ ]:
from langchain.chains import MapReduceDocumentsChain

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

Let's run this chain for the first 10 documents:


In [ ]:
print(map_reduce_chain.run(docs[:10]))

Looks like a decent summary of the main topics!

In the [next notebook](03-document-qa.ipynb), we will go one step further and ask the LLM more specific questions related to a specific document.


<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
